# Data Collection

This notebook contains the code necessary to extract the relevant data from the GHCN climate .tar archive. This dataset is too large to extract in its entirety onto my machine (>50GB) and so only the relevant files are extracted. This process is computationally expensive and so is kept separate from the overall Data Wrangling which occurs in notebook 02-rrp-data-wrangling.

The notebook is structured as follows:
1. Import relevant packages
2. Write function to import relevant weather data
3. Extract files of stations within 50km of conflict incidents

**IMPORTANT NOTE:** 

**Sections 1 and 3 can be run independently in order to allow effective running of the following notebooks (02, 03, etc.). Section 2 is superfluous to the actual running of the project and is included only as proof of progress.**

## 1. Importing packages

In [1]:
# Note to self: use kernel "Python-geoTiledb208" on old Macbook Pro
import pandas as pd
import numpy as np
import tarfile
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, Polygon 
import pyproj
import contextily as ctx
import warnings

In [2]:
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

In [ ]:
# use this only when working on old MacBook Pro
#pyproj.datadir.set_data_dir("/Users/richard/.conda/envs/geo/share/proj/")

## 2. Writing Functions to Extract Relevant Files From GHCN .tar Archive

*This section is included only as proof of progress and is not essential to the running of the project.*

**Lesson learned:** These functions were written before I was aware of the possibilities offered by spatial joins. By executing a spatial join of the climate stations on the buffered conflict data (done in notebook 02-rrp-data-wrangling) I was able to get all of the stations relevant to my analysis (i.e. those within 50km of any conflict) in one go. This proved more efficient than the functions I had written.

### Before starting:
From the documentation of the GHCN dataset we learn the following:

- There are two relevant datasets for our analysis:
 1. The GHCN_daily dataset with daily climate measures from each station
 2. The stations dataset with metadata (name, location, etc.) of each station
 

- Both of these datasets are stored in .dly format
- The .dly format is a *fixed-width format* meaning that each column has a fixed width of characters. 
- As such we will need to specify the column widths and column labels in order to get this looking like something we can use.

### Approach
In this section, we write two functions to import the relevant GHCN data from the .tar archive:

1. read_ghcn_incl_stationID: a general function that imports all .dly files from a .tar archive using the corresponding data headers and column specs.
2. find_stations: a function that finds and extract stations with a specified radius r of a specific conflict incident.

### 2.1. read_ghcn_incl_stationID

This is a general function that imports all .dly files from a .tar archive using the corresponding data headers and column specs. I have based my work in this section on this very useful GitLab [snippet](https://gitlab.com/snippets/1838910).

In [37]:
# Setting up specs for station data

data_header_names = [
    "STATION ID",
    "YEAR",
    "MONTH",
    "ELEMENT"
]

data_header_col_specs = [
    (0,  11),
    (11, 15),
    (15, 17),
    (17, 21)
]

data_header_dtypes = {
    "ID": str,
    "YEAR": int,
    "MONTH": int,
    "ELEMENT": str
}

data_col_names = [[
    "VALUE" + str(i + 1),
    "MFLAG" + str(i + 1),
    "QFLAG" + str(i + 1),
    "SFLAG" + str(i + 1)]
    for i in range(31)
]

# Join sub-lists
data_col_names = sum(data_col_names, [])

data_replacement_col_names = [[
    ("VALUE", i + 1),
    ("MFLAG", i + 1),
    ("QFLAG", i + 1),
    ("SFLAG", i + 1)]
    for i in range(31)
]

# Join sub-lists
data_replacement_col_names = sum(data_replacement_col_names, [])

data_replacement_col_names = pd.MultiIndex.from_tuples(
    data_replacement_col_names,
    names=['VAR_TYPE', 'DAY'])

data_col_specs = [[
    (21 + i * 8, 26 + i * 8),
    (26 + i * 8, 27 + i * 8),
    (27 + i * 8, 28 + i * 8),
    (28 + i * 8, 29 + i * 8)]
    for i in range(31)
]

data_col_specs = sum(data_col_specs, [])

data_col_dtypes = [{
    "VALUE" + str(i + 1): int,
    "MFLAG" + str(i + 1): str,
    "QFLAG" + str(i + 1): str,
    "SFLAG" + str(i + 1): str}
    for i in range(31)
]

data_header_dtypes.update({k: v for d in data_col_dtypes for k, v in d.items()})

In [1]:
def read_ghcn_incl_stationID(filename,
                        variables=None, include_flags=False,
                        dropna='all'):
    """Reads in all data from a GHCN .dly data file

    :param filename: path to file
    :param variables: list of variables to include in output dataframe
        e.g. ['TMAX', 'TMIN', 'PRCP']
    :param include_flags: Whether to include data quality flags in the final output
    :returns: Pandas dataframe
    """

    df = pd.read_fwf(
        filename,
        colspecs=data_header_col_specs + data_col_specs,
        names=data_header_names + data_col_names,
        index_col=data_header_names, #data_header_names[0]='ID'
        dtype=data_header_dtypes
        )

    if variables is not None:
        df = df[df.index.get_level_values('ELEMENT').isin(variables)]

    df.columns = data_replacement_col_names

    if not include_flags:
        df = df.loc[:, ('VALUE', slice(None))]
        df.columns = df.columns.droplevel('VAR_TYPE')

    df = df.stack(level='DAY').unstack(level='ELEMENT')

    if dropna:
        df.replace(-9999.0, np.nan, inplace=True)
        df.dropna(how=dropna, inplace=True)
    
    df.reset_index(level='STATION ID', inplace=True)
    
    # replace the entire index with the date.
    # This loses the station ID index column!
    # This will usuall fail if dropna=False, since months with <31 days
    # still have day=31 columns
    df.index = pd.to_datetime(
        df.index.get_level_values('YEAR') * 10000 +
        df.index.get_level_values('MONTH') * 100 +
        df.index.get_level_values('DAY'),
        format='%Y%m%d')
    
    return df

### 2.2. find_stations
This function finds stations within specified radius of a conflict ID and saves the corresponding .dly files as .csv in the specified directory.

In [39]:
# This function finds stations within specified radius of a conflict ID 
# and saves the corresponding .dly files as .csv in the specified directory

def find_stations(conflict_id, r, directory):
    """Extract .dly files of GHCN weather stations within r radius of conflict incident.
    
    Parameters:
    conflict_id: unique ID of conflict of interest, provided as integer. 
    r: radius within which to search for weather stations, in meters, provided as integer.
    directory: path to existing directory where extracted .dly file will be saved as .csv, excluding the file name and extension.
    
    The function will save an additional .csv file in the specified directory containing only the IDs of the stations. This file will be titled radius_xx_station_IDs.csv, where xx is the radius specified in meters.
    
    """
    
    # 1. Get long and lat of conflict incident as a Geometry Point
    conflict_point = gdf_conflict_mtr.loc[conflict_id, ['geometry']]['geometry']

    # 2. Create buffer using r
    buffer = conflict_point.buffer(r)

    # 3. Generate list of weather station IDs in radius
    neighbours = gdf_stations_mtr["geometry"].intersection(buffer)
    neighbours = neighbours[~(neighbours.is_empty | neighbours.isna())] # remove null/missing values
    
    stations_list = []
    for i in neighbours.index:
        stations_list.append(i)
    
    df_stations = pd.DataFrame(stations_list)
    path_stationIDs = directory + 'radius_' + str(r) + '_stationIDs.csv'
    df_stations.to_csv(path_stationIDs)
    
    # 4. Extract respective .dly files to path
    with tarfile.open("/Users/data_science/Desktop/datasets/GHCNdaily/ghcnd_all.tar.gz", "r:*") as tar: 

        for ID in stations_list:
            # extract corresponding file into dataframe
            # add TRY-EXCEPT clause to account for missing or incorrect filenames
            try:
                dly_path = 'ghcnd_all/' + ID + '.dly'
                filepath = tar.extractfile(dly_path)
                df_ID = read_ghcn_incl_stationID(filepath)
            except KeyError:
                print('A KeyError was raised for station: ' + ID +'. Either the filepath is incorrect or the corresponding station file is missing.')

            # turn dataframe into unique csv
            path = directory + ID + '.csv'
            df_ID.to_csv(path)


# 3. Extract Files of Stations Within 50km of Conflict Incidents

After writing the functions in section 2, I discovered the possibilities offered by performing a spatial join. This method proved more effective and efficient than the functions I wrote and so the approach is as follows:

Notebook **02-rrp-data-wrangling**
1. Creates geodataframes of the station metadata and the conflict data
2. Executes a spatial join of the stations with the **buffered** conflict data (buffer = 50km)
3. Extracts the unique station_IDs from that spatial join as a list

Notebook **01-rrp-data-collection (i.e. THIS notebook)**

4. Imports the list of unique station_IDs
5. Extracts the .dly files of those stations
6. Exports them as local csv files

Notebook **02-rrp-data-wranging**

7. Imports these csv files
8. Combines them into df_weather
9. Wrangles the data into workable shape

In [55]:
# import list of unique station IDs within 50km of conflicts as df
df_stationIDs_50k = pd.read_csv('/Users/data_science/Desktop/springboard_repo/capstones/capstone-two/data/interim/stationIDs_50k.csv', index_col=0)
df_stationIDs_50k.head()

,STATION ID
0,AE000041196
1,AEM00041194
2,AF000040930
3,AFM00040938
4,AFM00040948


In [57]:
# convert to list
list_stations_buf50k = []

for ID in df_stationIDs_50k['STATION ID']:
    list_stations_buf50k.append(ID)

In [62]:
# check length to verify
len(list_stations_buf50k)

10455

In [63]:
# use list to index into .tar archive and extract only the relevant station .dly files
# export those to local .csv files
%%time

with tarfile.open("/Users/data_science/Desktop/datasets/GHCNdaily/ghcnd_all.tar.gz", "r:*") as tar: 

    for ID in list_stations_buf50k:
        # extract corresponding file into dataframe
        # add TRY-EXCEPT clause to account for missing or incorrect filenames
        try:
            dly_path = 'ghcnd_all/' + ID + '.dly'
            filepath = tar.extractfile(dly_path)
            df_ID = read_ghcn_incl_stationID(filepath)
        except KeyError:
            print('A KeyError was raised for station: ' + ID +'. Either the filepath is incorrect or the corresponding station file is missing.')

        # turn dataframe into unique csv
        directory = '/Users/data_science/Desktop/springboard_repo/capstones/capstone-two/data/raw/extracted_stati§
        path = directory + ID + '.csv'
        df_ID.to_csv(path)

A KeyError was raised for station: PKM00041529. Either the filepath is incorrect or the corresponding station file is missing.
CPU times: user 20h 37min 1s, sys: 26min 33s, total: 21h 3min 35s
Wall time: 21h 53min 57s
